# Подготовка

In [1]:
import pandas as pd
from pandas.compat import StringIO

In [2]:
import numpy as np
from tqdm import tqdm_notebook

In [3]:
df_ratings = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/17 занятие - коллаборативная фильтрация/ml-1m/ratings.dat', header=None, sep='::', engine='python')
df_movies = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/17 занятие - коллаборативная фильтрация/ml-1m/movies.dat', header=None, sep='::', engine='python')

In [4]:
df_movies.columns=['movieId', 'title', 'genres']
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_ratings.columns=['userId','movieId','rating','timestamp']
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
df = pd.merge(df_ratings, df_movies, on='movieId')

In [7]:
del df['timestamp']
del df['genres']

# Построение модели

In [8]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
from surprise.model_selection import KFold

In [10]:
kfold = KFold(5)

In [12]:
df_for_surpise = df_ratings[['userId', 'movieId', 'rating']]

In [13]:
df_for_surpise.columns = ['uid', 'iid', 'rating']

In [14]:
df_for_surpise.head()

,uid,iid,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [15]:
reader = Reader(rating_scale=(0.5, 5))

In [16]:
dataset = Dataset.load_from_df(df_for_surpise, reader)

In [17]:
from surprise import KNNWithMeans

In [18]:
algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [19]:
from surprise.model_selection import cross_validate

In [20]:
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8605  0.8566  0.8569  0.8583  0.8577  0.8580  0.0014  
MAE (testset)     0.6738  0.6710  0.6713  0.6717  0.6713  0.6718  0.0010  
Fit time          27.04   31.06   33.42   32.51   27.59   30.32   2.58    
Test time         67.96   84.79   76.25   69.50   72

{'test_rmse': array([0.86052017, 0.85663543, 0.85687659, 0.85827613, 0.85768885]),
 'test_mae': array([0.67383267, 0.67100172, 0.67125583, 0.6717133 , 0.67126793]),
 'fit_time': (27.037880897521973,
  31.064035177230835,
  33.422778844833374,
  32.506141901016235,
  27.592067003250122),
 'test_time': (67.95606517791748,
  84.78920602798462,
  76.25480604171753,
  69.50158905982971,
  72.40942525863647)}

### Нужное значение RMSE получено (0.86 < 0.87), но интересно изучить другие модели (вдруг RMSE будет ниже)

# Исследование других методов

In [21]:
from surprise import SVD
algo_SVD = SVD()
cross_validate(algo_SVD, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8724  0.8748  0.8739  0.8735  0.8755  0.8740  0.0011  
MAE (testset)     0.6848  0.6868  0.6868  0.6853  0.6882  0.6864  0.0012  
Fit time          58.54   56.02   58.30   58.43   64.47   59.15   2.82    
Test time         3.37    2.95    3.09    3.52    3.47    3.28    0.22    


{'test_rmse': array([0.87235526, 0.87480749, 0.87389085, 0.87350613, 0.87549663]),
 'test_mae': array([0.68477469, 0.68676162, 0.68675176, 0.68531348, 0.68818524]),
 'fit_time': (58.544809103012085,
  56.0221381187439,
  58.301856994628906,
  58.43184304237366,
  64.4663679599762),
 'test_time': (3.3682339191436768,
  2.94895601272583,
  3.0940051078796387,
  3.521862745285034,
  3.4665119647979736)}

In [22]:
from surprise import NMF
algo_NMF = NMF()
cross_validate(algo_NMF, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9216  0.9142  0.9135  0.9153  0.9178  0.9165  0.0029  
MAE (testset)     0.7275  0.7222  0.7222  0.7234  0.7258  0.7242  0.0021  
Fit time          64.01   62.93   51.48   57.08   62.58   59.62   4.73    
Test time         3.01    2.65    2.60    3.99    2.28    2.91    0.59    


{'test_rmse': array([0.92156186, 0.9141939 , 0.91349787, 0.9152659 , 0.91778312]),
 'test_mae': array([0.7274798 , 0.72223311, 0.72221613, 0.72344231, 0.72580277]),
 'fit_time': (64.0071930885315,
  62.93132972717285,
  51.480660915374756,
  57.07799291610718,
  62.57813310623169),
 'test_time': (3.0108439922332764,
  2.6526901721954346,
  2.598159074783325,
  3.990407943725586,
  2.2759621143341064)}

In [23]:
from surprise import NormalPredictor
algo_NormalPredictor = NormalPredictor()
cross_validate(algo_NormalPredictor, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5126  1.5118  1.5132  1.5127  1.5095  1.5119  0.0013  
MAE (testset)     1.2114  1.2096  1.2116  1.2105  1.2077  1.2102  0.0014  
Fit time          1.43    1.67    1.60    1.70    1.55    1.59    0.10    
Test time         2.87    2.67    3.03    3.94    2.87    3.08    0.44    


{'test_rmse': array([1.51256828, 1.51178275, 1.51315807, 1.51266259, 1.5094597 ]),
 'test_mae': array([1.21138281, 1.20963334, 1.21159002, 1.21045391, 1.20770256]),
 'fit_time': (1.431995153427124,
  1.6684308052062988,
  1.5973927974700928,
  1.7020001411437988,
  1.549185037612915),
 'test_time': (2.8741848468780518,
  2.67297101020813,
  3.0309128761291504,
  3.9382379055023193,
  2.8742988109588623)}

In [24]:
from surprise import BaselineOnly
algo_BaselineOnly = BaselineOnly()
cross_validate(algo_BaselineOnly, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9094  0.9079  0.9100  0.9075  0.9084  0.9086  0.0009  
MAE (testset)     0.7201  0.7201  0.7200  0.7188  0.7183  0.7195  0.0007  
Fit time          2.63    2.78    2.79    2.98    3.24    2.89    0.21    
Test time         3.73    3.71    2.83    2.75    3.23    3.25    0.42    


{'test_rmse': array([0.90941602, 0.90790747, 0.90996649, 0.9075165 , 0.90835023]),
 'test_mae': array([0.72008693, 0.72005832, 0.7200134 , 0.71877955, 0.71832641]),
 'fit_time': (2.6301660537719727,
  2.7848258018493652,
  2.7916061878204346,
  2.982697010040283,
  3.2387850284576416),
 'test_time': (3.728868246078491,
  3.7086100578308105,
  2.8298709392547607,
  2.7505369186401367,
  3.225011110305786)}

In [25]:
from surprise import KNNBasic
algo_KNNBasic = KNNBasic()
cross_validate(algo_KNNBasic, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9242  0.9219  0.9218  0.9235  0.9217  0.9226  0.0010  
MAE (testset)     0.7281  0.7268  0.7267  0.7279  0.7268  0.7273  0.0006  
Fit time          42.35   40.23   34.75   40.22   35.40   38.59   2.98    
Test time         164.29  156.28  133.96  161.52  163.59  155.93  11.34   


{'test_rmse': array([0.92416658, 0.92193215, 0.92179883, 0.92352458, 0.92170048]),
 'test_mae': array([0.72810868, 0.72677075, 0.72668786, 0.72793014, 0.72684684]),
 'fit_time': (42.35262107849121,
  40.23359489440918,
  34.746777057647705,
  40.21861290931702,
  35.39767789840698),
 'test_time': (164.28721594810486,
  156.28431272506714,
  133.95750498771667,
  161.5218367576599,
  163.5898208618164)}

In [26]:
from surprise import KNNWithZScore
algo_KNNWithZScore = KNNWithZScore()
cross_validate(algo_KNNWithZScore, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9332  0.9296  0.9307  0.9282  0.9339  0.9311  0.0022  
MAE (testset)     0.7391  0.7366  0.7365  0.7349  0.7401  0.7374  0.0019  
Fit time          46.95   43.11   45.50   48.33   45.63   45.90   1.73    
Test time         343.10  174.38  188.88  184.54  168.60  211.90  65.99   


{'test_rmse': array([0.93324036, 0.92960987, 0.93072116, 0.92820344, 0.93390742]),
 'test_mae': array([0.73909832, 0.73661642, 0.73645805, 0.73494909, 0.74008254]),
 'fit_time': (46.94733476638794,
  43.110575914382935,
  45.504685163497925,
  48.329402685165405,
  45.63011407852173),
 'test_time': (343.0994551181793,
  174.37753200531006,
  188.88455200195312,
  184.53735303878784,
  168.59891605377197)}

In [27]:
from surprise import KNNBaseline
algo_KNNBaseline = KNNBaseline()
cross_validate(algo_KNNBaseline, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8945  0.8980  0.8934  0.8949  0.8940  0.8950  0.0016  
MAE (testset)     0.7058  0.7089  0.7057  0.7065  0.7057  0.7065  0.0012  
Fit time          36.41   34.29   271.02  34.85   36.81   82.68   94.18   
Test time         143.38  142.58  145.20  144.15  145.47  144.16  1.08    


{'test_rmse': array([0.89451591, 0.89795335, 0.89338809, 0.89490281, 0.89404356]),
 'test_mae': array([0.7058366 , 0.70887894, 0.70565062, 0.7064604 , 0.70565326]),
 'fit_time': (36.405524015426636,
  34.29443001747131,
  271.02236223220825,
  34.85339689254761,
  36.8053560256958),
 'test_time': (143.38321590423584,
  142.58165097236633,
  145.1959228515625,
  144.15478491783142,
  145.46718001365662)}

In [28]:
from surprise import SlopeOne
algo_SlopeOne = SlopeOne()
cross_validate(algo_SlopeOne, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9065  0.9075  0.9065  0.9061  0.9064  0.9066  0.0005  
MAE (testset)     0.7145  0.7153  0.7145  0.7146  0.7139  0.7146  0.0005  
Fit time          21.07   19.69   20.06   17.46   22.18   20.09   1.58    
Test time         73.32   74.48   71.83   73.48   73.18   73.26   0.85    


{'test_rmse': array([0.90653243, 0.90749662, 0.90654733, 0.90607228, 0.90644747]),
 'test_mae': array([0.71452209, 0.71534627, 0.71452301, 0.71461409, 0.71386641]),
 'fit_time': (21.074519872665405,
  19.688817977905273,
  20.0635929107666,
  17.46235680580139,
  22.184916973114014),
 'test_time': (73.32280015945435,
  74.48218178749084,
  71.8323872089386,
  73.4793860912323,
  73.1797239780426)}

In [29]:
from surprise import CoClustering
algo_CoClustering = CoClustering()
cross_validate(algo_CoClustering, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9151  0.9194  0.9144  0.9175  0.9144  0.9162  0.0020  
MAE (testset)     0.7177  0.7200  0.7172  0.7197  0.7167  0.7183  0.0014  
Fit time          24.67   21.37   18.32   18.59   18.34   20.26   2.49    
Test time         4.59    3.83    2.97    3.83    3.86    3.81    0.51    


{'test_rmse': array([0.91510846, 0.91944425, 0.91444317, 0.91753186, 0.9144457 ]),
 'test_mae': array([0.7177048 , 0.72002633, 0.71716198, 0.71968267, 0.7166916 ]),
 'fit_time': (24.671598196029663,
  21.36938977241516,
  18.323966026306152,
  18.594406127929688,
  18.34129285812378),
 'test_time': (4.589854001998901,
  3.8322207927703857,
  2.9660279750823975,
  3.8259048461914062,
  3.857079029083252)}

In [30]:
algo_KNNWithMeans_cosine = KNNWithMeans(k=40, sim_options={'name': 'cosine', 'user_based': False})
cross_validate(algo_KNNWithMeans_cosine, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8941  0.8926  0.8968  0.8943  0.8924  0.8940  0.0016  
MAE (testset)     0.7024  0.7016  0.7046  0.7031  0.7015  0.7026  0.0011  
Fit time          22.69   22.54   22.90   22.50   22.66   22.66   0.14    
Test time         67.13   67.26   68.79   64.76   66.30   66.85   1.32    


{'test_rmse': array([0.89407495, 0.89262383, 0.89683266, 0.89430667, 0.89237636]),
 'test_mae': array([0.70242116, 0.70155612, 0.70455041, 0.70314443, 0.70146735]),
 'fit_time': (22.685994148254395,
  22.544761180877686,
  22.901756048202515,
  22.49994921684265,
  22.659252882003784),
 'test_time': (67.13219499588013,
  67.25701093673706,
  68.78685283660889,
  64.76094889640808,
  66.29925608634949)}

In [31]:
algo_KNNWithMeans_pearson = KNNWithMeans(k=40, sim_options={'name': 'pearson', 'user_based': False})
cross_validate(algo_KNNWithMeans_pearson, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8877  0.8884  0.8889  0.8872  0.8893  0.8883  0.0007  
MAE (testset)     0.6970  0.6970  0.6972  0.6960  0.6977  0.6970  0.0005  
Fit time          29.67   30.07   30.08   29.79   33.24   30.57   1.34    
Test time         65.10   66.13   65.18   66.08   66.36   65.77   0.52    


{'test_rmse': array([0.88770816, 0.88837258, 0.88887016, 0.88724057, 0.88927435]),
 'test_mae': array([0.69703628, 0.69700726, 0.69718957, 0.69601926, 0.69768233]),
 'fit_time': (29.674993753433228,
  30.074726819992065,
  30.082863092422485,
  29.787450075149536,
  33.23699498176575),
 'test_time': (65.1019937992096,
  66.12518405914307,
  65.17915892601013,
  66.08287477493286,
  66.36296224594116)}

In [32]:
algo_KNNWithMeans_msd = KNNWithMeans(k=40, sim_options={'name': 'msd', 'user_based': False})
cross_validate(algo_KNNWithMeans_msd, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8867  0.8839  0.8833  0.8871  0.8869  0.8856  0.0016  
MAE (testset)     0.6951  0.6938  0.6932  0.6963  0.6963  0.6949  0.0013  
Fit time          12.79   11.75   14.17   12.06   14.40   13.04   1.08    
Test time         68.27   81.44   73.80   76.59   88.49   77.72   6.87    


{'test_rmse': array([0.88669487, 0.88394479, 0.88330788, 0.88712055, 0.88685642]),
 'test_mae': array([0.69506237, 0.69375808, 0.69319302, 0.69634224, 0.69634544]),
 'fit_time': (12.793275117874146,
  11.751168012619019,
  14.172672986984253,
  12.063903093338013,
  14.403142929077148),
 'test_time': (68.26624798774719,
  81.44155097007751,
  73.79881715774536,
  76.59076499938965,
  88.48763799667358)}

# Итог

Лучший результат получен для метода:
KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})

Но также интерес вызывает алгоритм BaselineOnly. Хотя RMSE для него не самое низкое, но время работы значительно ниже, чем для других методов. Возможно, в каких-то ситуациях это может быть важно.